# Exploring Twitter Data with Vector Databases and RAG Systems

This tutorial introduces the creation of a vector database and the use of a retrieval-augmented generation (RAG) system to explore Twitter data interactively. 

Please check [LBSocial](www.lbsocial.net)  on how to collect Twitter data. 

## Set up a Database and API Keys

Create a [MongoDB](www.mongodb.com) cluster and store the connection string in a safe place, such as AWS Secrets Manager. 
- key name: `connection_string`
- key value: <`the connection string`>, you need to type the password
- secret name: `mongodb`


You also need to purchase and your [oepnai](https://openai.com/) api key in AWS Secrets Manager:
- key name: `api_key`
- key value: <`your openai api key`>
- secret name: `openai`

## Install Python Libraries

- pymongo: manage the MongoDB database
- openai: create embeddings and resonpses 

In [1]:
pip install pymongo openai -q

Note: you may need to restart the kernel to use updated packages.


## Secrets Manager Function

In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [3]:
import pymongo
from pymongo import MongoClient
import json
import re
import os
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)


mongodb_connect = get_secret('mongodb')['connection_string']


## Connect to the MongoDB cluster

In [4]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection

## Utility Funcitons

- the `clean_tweet` function removes URLs in tweets
- the `get_embedding` function use openai to create tweet embeddings
- the `vector_search` function return relevent tweets based on a query

In [5]:
def clean_tweet(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)

In [6]:
embedding_model= 'text-embedding-3-small'

def get_embedding(text):

    try:
        embedding = client.embeddings.create(input=text, model=embedding_model).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

In [7]:
def vector_search(query):

    query_embedding = get_embedding(query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."
    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "tweet_vector",
                "queryVector": query_embedding,
                "path": "tweet.embedding",
                "numCandidates": 1000,  # Number of candidate matches to consider
                "limit": 10  # Return top 10 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "tweet.text": 1 # return tweet text

            }
        }
    ]

    results = tweet_collection.aggregate(pipeline)
    return list(results)

## Tweets Embedding 

For more about text embeddings please read [Introducing text and code embeddings](https://openai.com/index/introducing-text-and-code-embeddings/)

In [8]:
from tqdm.auto import tqdm
tweets = tweet_collection.find()

for tweet in tqdm(list(tweets)):
    try:
        tweet_embedding = get_embedding(clean_tweet(tweet['tweet']['text']))
    #     print(tweet_embedding)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet']['id']},
            {"$set":{'tweet.embedding':tweet_embedding}}
        )
    except:
        print(f"""error in embedding tweet {tweet['tweet']['id']}""")
        pass


  0%|          | 0/99 [00:00<?, ?it/s]

## Create a Vector Index

For more about the MognoDB Vector database, please read [What are Vector Databases?](https://www.mongodb.com/resources/basics/databases/vector-databases)
This code creates a vector index following the [MongoDB official document](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#std-label-avs-types-vector-search).

In [9]:
# Create your index model, then create the search index

from pymongo.operations import SearchIndexModel
import time

search_index_model = SearchIndexModel(
  definition={
  "fields": [
    {
      "type": "vector",
      "path": "tweet.embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
},
  name="tweet_vector",
  type="vectorSearch"

)
result = tweet_collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None

if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(tweet_collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):

    break
  time.sleep(5)

print(result + " is ready for querying.")

New search index named tweet_vector is building.
Polling to check if the index is ready. This may take up to a minute.
tweet_vector is ready for querying.


In [14]:
user_query = 'united states voter fraud in foreign elections'

for tweet in vector_search(user_query):
    print(tweet['tweet']['text'])

RT @TheIntelSCIF: U.S. TAXPAYERS funding massive GLOBAL ELECTION FRAUD CARTEL connected to over 100+ countries called A-WEB, tied to USAID,…
@DefiantLs Its not about the voters Abigail, its about the fake votes - attached is the voter recap that proves election and fake votes.  Enjoy https://t.co/9UpFwcwu1O
RT @jjauthor: Will we ever get the OBVIOUS and most damaging problem addressed? When the Election Fraud will be stopped? 
No, because they…
RT @RealLindellTV: 🚨 BREAKING: Marly Hornick SUES NY Board of Elections and Letitia James for Election Fraud 🚨

@MarlyHornik of @RealAmeric…
@Edward26553035 @DMetronomeENT @TheThe1776 And again if they rigged the election. Why did Donald win in 2024
RT @bloody_media: What the hell is happening in West Bengal? 😳😨 

A guy was caught in Nadia with over 10,000 voter cards! 

Was this a plan…
@nedryun They steal the election and then go after everyone on the other side.
RT @Honcalebamisi: This is unique,instead of protesting the results of an electio

## Retrieval-Augmented Generation (RAG) 

For more about RAG, please read [Retrieval-Augmented Generation (RAG) with Atlas Vector Search](https://www.mongodb.com/docs/atlas/atlas-vector-search/rag/#std-label-avs-rag).

In [15]:
from openai import OpenAI

delimiter = '###'
chat_model = 'gpt-4o'
temperature = 0

chat_history = [{"role": "system", "content": """ you are a chatbot, answer user questions based on the returned tweets."""}]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})
    
    tweets = vector_search(prompt)
    chat_history.append({"role": "system", "content": f" here the returned tweets delimited by {delimiter}{tweets}{delimiter}"})

    response = client.chat.completions.create(
        model=chat_model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [16]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  election fraud


Chatbot: The tweets mention various claims and actions related to election fraud, including:

1. Allegations of fake votes being part of election results, with a user claiming to have a voter recap as proof.
2. A tweet suggesting that U.S. taxpayers are funding a global election fraud cartel, connected to over 100 countries, via an organization called A-WEB and tied to USAID.
3. A question about why Donald Trump might have won an election if it was allegedly rigged.
4. Marly Hornick suing the New York Board of Elections and Letitia James over election fraud.
5. Concerns being raised about when election fraud will be addressed and suggesting that the issue is not being solved.
6. Accusations of stolen elections and subsequent targeting of those on the opposing side.
7. Mention of protests not against election results but against the election itself.
8. Concerns about the potential impact of ICE's quota system on public behavior during elections.
9. Discussion on the reasons behind Presi

You:  election news


Chatbot: Here are some of the latest tweets related to election news:

1. A tweet expresses anticipation for further developments in the election, encouraging not to "peak too early."
2. Puerto Rico Governor Jenniffer González-Colón was reportedly in New Jersey collaborating with political figures there.
3. Marly Hornick has sued the New York Board of Elections and Letitia James over election fraud.
4. Mention of right-wing efforts leading up to the 2024 election.
5. A claim about fake votes attached to a voter recap document.
6. Encouragement for Zohran supporters to participate actively in the election despite favorable polls.
7. No election is currently taking place in Tanzania, with a tense atmosphere as citizens demand change.
8. In New Jersey (N.J.), there seems to be evidence of increased GOP voter turnout.
9. News about five of Maura Healey’s cabinet secretaries resigning, setting a record for a first-time governor.
10. Advocacy for protests on election day, with concerns about

You:  conspiracy theories


Chatbot: Here are some tweets related to conspiracy theories:

1. There is a claim that U.S. taxpayers are funding a global election fraud cartel connected to over 100 countries, named A-WEB, supposedly tied to USAID.
2. A tweet discussing why President Trump never conceded the 2020 election links to a source that may provide more information.
3. Mention of efforts by rightwingers prior to the 2024 election.
4. Questions about why Donald Trump would win in 2024 if elections are allegedly rigged.
5. Allegations of collaboration between Team Cuomo and Team Trump to discredit a political opponent using substantial resources.
6. Claims regarding ICE's quota system as a means to control the population, potentially impacting elections.
7. Accusations of elections being stolen and subsequent targeting of individuals on the opposing side.
8. Discussion on AIPAC's political leanings and financial influence in elections, including during Democratic primaries.
9. Controversy over Pam Bondi's acti

You:  quit


Chatbot: Goodbye!


## Reference

- *“Introducing Text and Code Embeddings.”* n.d. OpenAI. Accessed October 31, 2024. https://openai.com/index/introducing-text-and-code-embeddings/.
- *“What Are Vector Databases?”* n.d. MongoDB. Accessed October 31, 2024. https://www.mongodb.com/resources/basics/databases/vector-databases.
